In [ ]:
import numpy as np
import random

In [ ]:
# Tic Tac Toe game board
board = np.zeros((3, 3))

# Possible actions (empty spaces)
actions = [(i, j) for i in range(3) for j in range(3)]


In [ ]:
# Function to check if a player has won
def check_win(board, player):
    # Check rows
    if np.any(np.all(board == player, axis=1)):
        return True
    # Check columns
    if np.any(np.all(board == player, axis=0)):
        return True
    # Check diagonals
    if np.all(np.diag(board) == player) or np.all(np.diag(np.fliplr(board)) == player):
        return True
    return False


In [ ]:
# Probability transition function
T = np.zeros((len(actions), 2, len(actions)))
for i, action in enumerate(actions):
    row, col = action
    for j, next_action in enumerate(actions):
        next_row, next_col = next_action
        if board[row][col] != 0 or board[next_row][next_col] != 0:
            # Invalid action (either current or next space is already occupied)
            T[i, :, j] = 0
        else:
            # Valid action
            if np.sum(board == 1) == np.sum(board == 2):
                # Opponent's turn (player plays as 2)
                T[i, 0, j] = 1/len(actions)
                board[row][col] = 1
                if check_win(board, 1):
                    # Opponent wins (invalidates further moves)
                    T[i, :, j] = 0
                board[row][col] = 0
            else:
                # Player's turn
                T[i, 1, j] = 1/len(actions)
                board[row][col] = 2
                if check_win(board, 2):
                    # Player wins (invalidates further moves)
                    T[i, :, j] = 0
                board[row][col] = 0


In [ ]:
# Arbitrary policy (randomly select an action from available actions)
policy = [random.choice(range(len(actions))) for _ in range(len(board.flatten()))]

In [ ]:
# Value iteration algorithm
gamma = 0.9
W = np.zeros(len(actions))
for t in range(100):
    W_prev = W.copy()
    for i, action in enumerate(actions):
        Ra = np.zeros(2)
        for j in range(len(actions)):
            Ra[0] += T[i, 0, j] * W_prev[j]
            Ra[1] += T[i, 1, j] * W_prev[j]
        W[i] = max(Ra + gamma * np.sum(T[i, 1, :] * W_prev))
    if np.allclose(W, W_prev):
        break


In [ ]:
# Optimal policy (select the action that maximizes the value function at each state)
opt_policy = [np.argmax(Ra + gamma * np.sum(T[i, 1, :] * W)) for i, Ra in enumerate(T[:, 0, :])]

In [ ]:
# Example game simulation using optimal policy
while True:
    print(board)
    if check_win(board, 1):
        print("Opponent wins")
        break
    elif check_win(board, 2):
        print("Player wins")
        break
    elif len(actions) == 0:
        print("Draw")
        break
    else:
        i = random.choice(range(len(actions)))
        action = actions[i]
        row, col = action
        print("Opponent plays at", action)
        board[row][col] = 1
        if check_win(board, 1):
            print(board)
            print("Opponent wins")
            break
        elif check_win(board, 2):
            print(board)
            print("Player wins")
            break
        elif len(actions) == 0:
            print(board)
            print("Draw")
            break
        else:
            i = opt_policy.index(max(opt_policy))
            opt_action = actions[i]
            opt_row, opt_col = opt_action
            print("Player plays at", opt_action)
            board[opt_row][opt_col] = 2
            actions.remove(opt_action)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Opponent plays at (0, 2)
Player plays at (0, 0)
[[2. 0. 1.]
 [0. 0. 0.]
 [0. 0. 0.]]
Opponent plays at (1, 1)
Player plays at (0, 1)
[[2. 2. 1.]
 [0. 1. 0.]
 [0. 0. 0.]]
Opponent plays at (2, 1)
Player plays at (0, 2)
[[2. 2. 2.]
 [0. 1. 0.]
 [0. 1. 0.]]
Player wins
